# ⛈️ ⛈️ Bayesian Optimization Based Control of Stormwater Systems ⛈️ ⛈️

In this notebook, we introduce an automated approach for control of stormwater systems.
Bayesian Optimization is a sample effecient approach for idenitifying the optima in problems that are hard or not easily solved using optimization frameworks.

Baysian optimization is numerical approach, which explores the entire solution space to indentiify the solution that minimizes the objective function. Though this is similar to how genetic algoritms works, the thing that makes it different is how it searches through the solution space. 

For example consider the problem where we are trying to find the valve position where we are trying to manage flooding.

Bayesian optimization initales tries random times and as more time happens it find the best solution for its problem.

![bayesianOPT](https://scontent-sin6-1.xx.fbcdn.net/v/t39.8562-6/240822002_854859678751297_4744004142950977863_n.gif?_nc_cat=107&ccb=1-5&_nc_sid=6825c5&_nc_ohc=lOlHdF4ge3sAX-up-ln&_nc_ht=scontent-sin6-1.xx&oh=00_AT-4N_OJqQDts9Hc0jFBdiWkEwx9a5QttRntt6hCVNySRg&oe=61DCF78C)

DO an example algorithm might be good to have. 

We would be using gamma as an example

Talk about what goes into gamme

In [20]:
import pystorms
import numpy as np
import pandas as pd
from GPyOpt.methods import BayesianOptimization
import matplotlib.pyplot as plt

In [2]:
# Gamma Scenario for the basins in series
def GammaData(actions, CTRL_ASSETS=4):

    # Initialize the scenario
    env = pystorms.Scenarios.gamma()
    done = False

    # Modify the logger function to store depths
    env.data_log["depthN"] = {}
    for i in np.linspace(1, CTRL_ASSETS, CTRL_ASSETS, dtype=int):
        env.data_log["depthN"][str(i)] = []

    # Dont log the params from upstream, we donot care about them
    for i in np.linspace(CTRL_ASSETS + 1, 11, 11 - CTRL_ASSETS, dtype=int):
        del env.data_log["flow"]['O' + str(i)]
        del env.data_log["flooding"][str(i)]

    # Simulate controlled response
    while not done:
        done = env.step(actions)

    # Return the logged params
    return env.data_log

In [3]:
# Objective Function
def f_loss(x):
    # GypOpt uses 2d array
    # pystorms requies 1d array
    x = x.flatten()

    # Simulate the response of control actions
    data = GammaData(x)

    # Convert to pandas dataframes
    depths = pd.DataFrame.from_dict(data["depthN"])
    flows = pd.DataFrame.from_dict(data["flow"])
    flooding = pd.DataFrame.from_dict(data["flooding"])

    # Compute loss - check the performance metric equation in Readme
    loss = 0.0

    # Flooding loss
    for node in flooding.keys():
        if flooding[node].sum() > 0.0:
            loss += 10 ** 4 * flooding[node].sum()

    # Flow deviation
    flows = flows.sub(4.0)
    flows[flows < 0.0] = 0.0
    loss += flows.sum().sum()

    # Prevent basins from storing water at the end.
    for i in depths.values[-1]:
        if i > 0.1:
            loss += i * 10 ** 3

    return loss

In [ ]:
# Read the parsed args
random_seed = 42
number_iter = 200

# Set the random seed
np.random.seed(random_seed)

# Create the domain
domain = []
for i in range(1, 5):
    domain.append({"name": "var_" + str(i), "type": "continuous", "domain": (0.0, 1.0)})


myBopt = BayesianOptimization(
    f=f_loss, domain=domain,
    model_type="GP",
    acquisition_type="EI",
)

myBopt.run_optimization(
    report_file=save_path + "_report.txt",
    max_iter=number_iter,
    verbosity=True,
    eps=0.005,
)

### Exercise 1
Update the f_loss function to use peformance metric and identify the optimal gate position 

Hint: `env.performance()` gives the overall performance from a model run

### Exercise 2
Using the control straegy presented above, find the control actions that minimize the performane metric in scenario epsilon.

![epsilon](epsilon.png)